# Lab 3: Silver Layer - Cleaned & Deduplicated Data

## 🎯 **Learning Objectives:**
- Read Bronze data như streaming source
- Clean và normalize stock trade data
- Deduplicate với watermarking
- Validate data quality
- Write cleaned data to Silver layer

## 📚 **Key Concepts:**
1. **Silver Layer**: Cleaned, validated, deduplicated data
2. **Data Cleaning**: Normalize, validate, enrich
3. **Deduplication**: Remove duplicates với watermark
4. **Watermarking**: Handle late-arriving data
5. **Upsert**: Merge new data với existing (Iceberg MERGE)


In [ ]:
# Setup
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

spark = SparkSession.builder \
    .appName("StreamingLakehouseSilver") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

BRONZE_TABLE_PATH = "/warehouse/bronze/trades"
SILVER_TABLE_PATH = "/warehouse/silver/trades"

print("🚀 Spark Session initialized for Silver Layer!")


## Exercise 1: Read Bronze như Streaming Source

### Key Point:
Silver layer reads từ Bronze như **streaming source**, không phải batch.
This allows real-time processing từ Bronze → Silver.


In [ ]:
# Read Bronze như Streaming Source
print("📥 Exercise 1: Read Bronze như Streaming Source")
print("=" * 60)

print(f"\n1️⃣ Reading Bronze layer as stream: {BRONZE_TABLE_PATH}")

bronze_stream = spark.readStream \
    .format("parquet") \
    .schema(spark.read.parquet(BRONZE_TABLE_PATH).schema) \
    .load(BRONZE_TABLE_PATH)

print("✅ Bronze stream created!")
print("\n📋 Bronze stream schema:")
bronze_stream.printSchema()

print("\n💡 Key insight:")
print("   Bronze → Silver: Streaming transformation")
print("   - Real-time processing")
print("   - Continuous data flow")
print("   - Low latency")


## Exercise 2: Data Cleaning & Normalization

### Cleaning Steps:
1. **Normalize timestamps**: Convert to proper timestamp type
2. **Validate prices**: Ensure positive prices
3. **Validate volumes**: Ensure positive volumes
4. **Normalize symbols**: Uppercase, trim whitespace
5. **Add computed fields**: total_value = price * volume
6. **Filter invalid data**: Remove records with nulls or invalid values


In [ ]:
# Data Cleaning & Normalization
print("🧹 Exercise 2: Data Cleaning & Normalization")
print("=" * 60)

print("\n1️⃣ Cleaning and normalizing data:")

cleaned_stream = bronze_stream \
    .filter(
        col("trade_id").isNotNull() &
        col("symbol").isNotNull() &
        col("price").isNotNull() &
        col("volume").isNotNull() &
        (col("price") > 0) &
        (col("volume") > 0)
    ) \
    .withColumn("symbol", upper(trim(col("symbol")))) \
    .withColumn("trade_type", upper(trim(col("trade_type")))) \
    .withColumn("event_timestamp", 
                coalesce(col("event_timestamp"), 
                        to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss'Z'"))) \
    .withColumn("total_value", col("price") * col("volume")) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .select(
        col("trade_id"),
        col("symbol"),
        col("price"),
        col("volume"),
        col("event_timestamp"),
        col("trade_type"),
        col("exchange"),
        col("total_value"),
        col("ingestion_timestamp")
    )

print("✅ Cleaning transformations applied!")
print("\n📋 Cleaned stream schema:")
cleaned_stream.printSchema()

print("\n💡 Cleaning operations:")
print("   ✅ Filter invalid records (nulls, negative prices/volumes)")
print("   ✅ Normalize symbols (uppercase, trim)")
print("   ✅ Normalize timestamps")
print("   ✅ Add computed fields (total_value)")
print("   ✅ Add ingestion timestamp")


## Exercise 3: Deduplication với Watermarking

### Key Concepts:
- **Watermark**: Threshold for late-arriving data
- **Deduplication**: Remove duplicate records
- **Event-time**: Use event_timestamp (not processing time)
- **Late data**: Data arriving after watermark is dropped

### Strategy:
- Watermark: 5 minutes (data older than 5 min is considered late)
- Deduplicate by: trade_id (unique identifier)


In [ ]:
# Deduplication với Watermarking
print("🔍 Exercise 3: Deduplication với Watermarking")
print("=" * 60)

print("\n1️⃣ Adding watermark (5 minutes):")

# Add watermark for late data handling
with_watermark = cleaned_stream \
    .withWatermark("event_timestamp", "5 minutes")

print("✅ Watermark added!")
print("   - Late data threshold: 5 minutes")
print("   - Data older than 5 min from latest event is dropped")

print("\n2️⃣ Deduplicating by trade_id:")

deduplicated = with_watermark \
    .dropDuplicates(["trade_id", "event_timestamp"])

print("✅ Deduplication applied!")
print("   - Deduplicate by: trade_id + event_timestamp")
print("   - Keeps first occurrence of each trade_id")

print("\n💡 Watermarking benefits:")
print("   ✅ Handles late-arriving data")
print("   ✅ Bounds state size (drops old data)")
print("   ✅ Enables Append mode với aggregations")


## Exercise 4: Write to Silver Layer

### Output Mode: Append
- With watermark, Append mode works
- Only new records are written
- Deduplication ensures no duplicates

### Note:
In production with Iceberg, use MERGE for upsert:
```python
.foreachBatch(lambda df, batchId:
    df.createOrReplaceTempView("updates")
    spark.sql("""
        MERGE INTO silver_trades AS t
        USING updates AS u
        ON t.trade_id = u.trade_id
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)
)
```


In [ ]:
# Write to Silver Layer
print("💾 Exercise 4: Write to Silver Layer")
print("=" * 60)

print(f"\n1️⃣ Writing to Silver layer: {SILVER_TABLE_PATH}")

silver_query = deduplicated \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", SILVER_TABLE_PATH) \
    .option("checkpointLocation", "/tmp/silver_checkpoint") \
    .trigger(processingTime='10 seconds') \
    .start()

print("✅ Silver streaming query started!")
print(f"   Writing to: {SILVER_TABLE_PATH}")
print(f"   Checkpoint: /tmp/silver_checkpoint")
print(f"   Output mode: Append (with watermark)")

print("\n💡 Silver layer characteristics:")
print("   ✅ Cleaned and validated data")
print("   ✅ Deduplicated")
print("   ✅ Normalized")
print("   ✅ Ready for analytics")

print("\n⚠️  Query is running. To stop: silver_query.stop()")


## Exercise 5: Verify Silver Data

### Compare Bronze vs Silver:
- Data quality improvements
- Deduplication results
- Schema differences


In [ ]:
# Verify Silver Data
print("🔍 Exercise 5: Verify Silver Data")
print("=" * 60)

time.sleep(15)  # Wait for data

print("\n1️⃣ Reading Silver data:")

try:
    silver_df = spark.read.parquet(SILVER_TABLE_PATH)
    bronze_df = spark.read.parquet(BRONZE_TABLE_PATH)
    
    print(f"✅ Silver data found!")
    print(f"   Bronze records: {bronze_df.count()}")
    print(f"   Silver records: {silver_df.count()}")
    
    print("\n2️⃣ Sample Silver data:")
    silver_df.show(10, truncate=False)
    
    print("\n3️⃣ Data quality check:")
    print(f"   Null trade_ids: {silver_df.filter(col('trade_id').isNull()).count()}")
    print(f"   Null prices: {silver_df.filter(col('price').isNull()).count()}")
    print(f"   Negative prices: {silver_df.filter(col('price') <= 0).count()}")
    
    print("\n4️⃣ Records by symbol:")
    silver_df.groupBy("symbol").count().orderBy(desc("count")).show()
    
    print("\n5️⃣ Price statistics:")
    silver_df.select("symbol", "price", "volume", "total_value").summary().show()
    
except Exception as e:
    print(f"⚠️  Error: {e}")
    print("   Make sure Silver query has processed some batches")


## Summary

### ✅ What we learned:
1. **Silver Layer**: Cleaned, validated, deduplicated data
2. **Streaming from Bronze**: Read Bronze như streaming source
3. **Data Cleaning**: Normalize, validate, enrich data
4. **Deduplication**: Remove duplicates với watermark
5. **Watermarking**: Handle late-arriving data
6. **Append Mode**: With watermark, Append mode works

### 🎯 Key Takeaways:
- **Silver = Clean**: Validated, normalized, deduplicated
- **Streaming Transform**: Bronze → Silver là streaming transformation
- **Watermarking**: Essential cho late data và state management
- **Ready for Analytics**: Silver data ready for Gold layer

### 🚀 Next Steps:
- Lab 4: Gold Layer (aggregations, features)
- Lab 5: Unified batch + streaming
